In [1]:
import ee
import geemap as emap

In [2]:
# Initialize and authenticate
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AX4XfWiejhD_uaO3eojVxf9wI7FoDE-H8U1rMzughSm_Bt5t2MuqChSb9yU

Successfully saved authorization token.


In [3]:
Map=emap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

# 1. Extract raster values from a single image (single time) 

##### 1.1. Extract rainfall  

In [4]:
# Extract total rainfall data for the period from 1/1/2018 to 30/1/2018
thang1=ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY").filterDate("2018-01-01","2018-01-30").sum()

In [5]:
# locations needed to get rainfall data
point=ee.FeatureCollection("users/miketu72/Test_point")

Map.centerObject(point,12)

Map.addLayer(point,{},"Location")

Map

Map(bottom=1851.0, center=[21.24149464251615, 105.49841825544672], controls=(WidgetControl(options=['position'…

In [6]:
# Extract total January rainfall  
extractedData=thang1.sampleRegions(collection=point, scale=1000,geometries =True).getInfo()

In [7]:
import pandas as pd
def dataFormat(inputDate):
    mlist=[]
    coord=[]
    for item in inputDate["features"]:
        mlist.append(item["properties"])
        coord.append(item["geometry"]["coordinates"])
    test=pd.DataFrame(mlist)
    test["coordiates"]=coord
    return test

In [8]:
df=dataFormat(extractedData)
df.head()

,Code,precipitation,coordiates
0,1,11.455535,"[105.46670593205242, 21.258631198688477]"
1,2,11.455535,"[105.48467223773481, 21.27659750437087]"
2,3,10.591261,"[105.5026385434172, 21.29456381005326]"
3,4,11.455535,"[105.47568908489362, 21.29456381005326]"
4,5,11.455535,"[105.45772277921122, 21.27659750437087]"


###### Extract mean temperature for a single image

The below dataset provides hourly global temperature as well as dozens of other variables. If you are more interested in this dataset, please see this [link](https://cds.climate.copernicus.eu/cdsapp#!/dataset/10.24381/cds.e2161bac?tab=overview).

The task below aims to extract mean of temperature in January 2017 from hourly temperature dataset.

In [9]:
# Extract mean temperature for the month of January 2017
tem=ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY").filterDate("2017-01-01","2017-01-30").select("temperature_2m").mean()

In [10]:
extractedData=tem.sampleRegions(collection=point, scale=1000,geometries =True).getInfo()

In [11]:
import pandas as pd
def dataFormat(inputDate):
    mlist=[]
    coord=[]
    for item in inputDate["features"]:
        mlist.append(item["properties"])
        coord.append(item["geometry"]["coordinates"])
    test=pd.DataFrame(mlist)
    test["coordiates"]=coord
    test["temperature_2m"]=test.temperature_2m-273.15
    return test

In [12]:
# This dataset contains all variables in the dataset such as temperature at 2m. Some values had the same because it those points all 
# in the same pixel
df=dataFormat(extractedData)
df.head()

,Code,temperature_2m,coordiates
0,1,18.205899,"[105.46670593205242, 21.258631198688477]"
1,2,18.205899,"[105.48467223773481, 21.27659750437087]"
2,3,18.205899,"[105.5026385434172, 21.29456381005326]"
3,4,18.205899,"[105.47568908489362, 21.29456381005326]"
4,5,18.205899,"[105.45772277921122, 21.27659750437087]"


###### Extract Landsat surface reflectance 

In [13]:
# Select Landsat 8 SR 
ls=ee.Image("LANDSAT/LC08/C01/T2_SR/LC08_127045_20170417")

In [14]:
extractedLS=ls.sampleRegions(collection=point, scale=30,geometries =True).getInfo()

In [15]:
import pandas as pd
def dataFormat(inputDate):
    mlist=[]
    coord=[]
    for item in inputDate["features"]:
        mlist.append(item["properties"])
        coord.append(tuple(item["geometry"]["coordinates"]))
    test=pd.DataFrame(mlist)
    test["coordiates"]=coord
    return test

In [16]:
df=dataFormat(extractedLS)

df.head()

,B1,B10,B11,B2,B3,B4,B5,B6,B7,Code,pixel_qa,radsat_qa,sr_aerosol,coordiates
0,6972,2875,2863,6931,6775,6861,7455,5491,3919,1,322,0,228,"(105.4665058457405, 21.257972172029646)"
1,7569,2877,2864,7526,7430,7532,8050,5838,3967,2,322,0,228,"(105.48216930891728, 21.27310711346932)"
2,7276,2875,2863,7181,6987,7029,7541,5402,3673,3,322,0,228,"(105.50480285025506, 21.295809232862858)"
3,6565,2872,2855,6496,6314,6350,6973,5077,3496,4,322,0,228,"(105.47645382927058, 21.294265541629027)"
4,6975,2872,2862,6882,6760,6817,7312,5549,4060,5,322,0,228,"(105.46077430709072, 21.274522256694354)"


# Extract time series raster values

**Extract daily rainfal data**

In [50]:
# Define time period from 1/1/2018 - 30/1/2018
daily=ee.ImageCollection("ECMWF/ERA5/MONTHLY").filterDate("2018-01-01","2020-12-31")

In [51]:
data=daily.getRegion(geometry=point, scale=1000).getInfo()

In [52]:
import datetime as dt
# This function aims to convert date code into human readable datetime. 
def timeConverter(date_code):
    start_date=dt.datetime(1970,1,1,0,0,0)
    hour_number=date_code/(60000*60)
    delta=dt.timedelta(hours=hour_number)
    end_date=start_date+delta
    return end_date
# This function to create a formated dataframe
import pandas as pd
def dataFormat(data):
    df=pd.DataFrame(data[1:], columns=data[0])
    df["time"]=[timeConverter(i) for i in df.time]
    return df
test=dataFormat(data)
test.head()

,id,longitude,latitude,time,mean_2m_air_temperature,minimum_2m_air_temperature,maximum_2m_air_temperature,dewpoint_2m_temperature,total_precipitation,surface_pressure,mean_sea_level_pressure,u_component_of_wind_10m,v_component_of_wind_10m
0,201801,105.520605,21.1688,2018-01-01,291.141846,282.706299,302.046173,287.676361,0.047356,101297.304688,101691.406250,-1.146670,0.221634
1,201802,105.520605,21.1688,2018-02-01,290.611786,282.927826,300.436768,285.954895,0.016731,101464.039062,101859.453125,-1.020320,0.045372
2,201803,105.520605,21.1688,2018-03-01,295.973450,286.264313,304.289337,292.002838,0.043602,100973.460938,101361.718750,-1.635398,0.918731
3,201804,105.520605,21.1688,2018-04-01,297.653320,288.470917,306.745422,293.704773,0.126332,100777.070312,101163.445312,-1.337478,0.478435
4,201805,105.520605,21.1688,2018-05-01,302.107788,296.628448,310.682587,298.212646,0.150164,100343.164062,100725.117188,-0.893623,0.739812
